# ANALYSIS.
## Producer price DB.

In [1]:
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")

import importlib
import src.analysis as an
import sys 
an = sys.path.append("\src\\analysis.py")

ModuleNotFoundError: No module named 'src'

IMPORT roducer price data set.

In [2]:
price= pd.read_csv("D:\ironhack\proyectos\_proy4\csv\\producer_price.csv")
price.sample()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Months Code,Months,Unit,Value,Flag,Flag Description
36370,PP,Producer Prices,804,Ukraine,5532,Producer Price (USD/tonne),21111.01b,"Meat of cattle with the bone, fresh or chilled...",2017,2017,7021,Annual value,USD,1366.5,A,Official figure


Clean Data Base.

In [4]:
# Drop columns.
price.drop(columns=["Domain", "Domain Code", "Element Code", "Element", "Year Code", "Months", "Months Code", "Flag", "Flag Description"],inplace=True)

In [5]:
# Change names.
price.rename(columns={"Area Code (M49)":"Area Code", "Item Code (CPC)": "Item Code"}, inplace= True)

In [7]:
def iterate_column_years (df, year):
    # iterate 2010
    x = {index:[row[0],row[1],row[2],row[3],row[6]] for index, row in df.iterrows() if row[4] == year}
    df = pd.DataFrame.from_dict(x, orient="index")
    df.rename(columns= {0: "Area Code", 1:"Area", 2:"Item Code", 3:"Item", 4:year}, inplace = True)
    df["Item"] = df["Item"].astype(object)
    df["Area"] = df["Area"].astype(object)
    df[year] = df[year].astype(object)
    
    return df.drop_duplicates(subset=['Area Code', 'Item Code'], keep='last')

Iteration throw each year and create column with year.

In [8]:
# iterate 2010
df2010 = iterate_column_years (price, 2010)

In [9]:
# iterate 2011
df2011 = iterate_column_years (price, 2011)

In [10]:
# iterate 2012
df2012 = iterate_column_years (price, 2012)

In [11]:
# iterate 2013
df2013 = iterate_column_years (price, 2013)

In [12]:
# iterate 2014
df2014 = iterate_column_years (price, 2014)

In [13]:
# iterate 2015
df2015 = iterate_column_years (price, 2015)

In [14]:
# iterate 2016
df2016 = iterate_column_years (price, 2016)

In [15]:
# iterate 2017
df2017 = iterate_column_years (price, 2017)

In [16]:
# iterate 2018
df2018 = iterate_column_years (price, 2018)

In [17]:
# iterate 2019
df2019 = iterate_column_years (price, 2019)

In [20]:
# iterate 2020
df2020 = iterate_column_years (price, 2020)

Drop columns of production and duplicates, and change types.

In [21]:
def clean2_df (df):
    df.drop(columns=["Year", "Value", "Unit"],inplace=True)
    df["Item Code"] = df["Item Code"].astype(object)
    df["Area Code"] = df["Area Code"].astype(object)
    # drop duplicates.
    df.drop_duplicates(inplace=True)
    return df

In [22]:
clean2_df (price)

,Area Code,Area,Item Code,Item
0,8,Albania,01341,Apples
22,8,Albania,0115,Barley
44,8,Albania,0231,"Hen eggs in shell, fresh"
66,8,Albania,0112,Maize (corn)
88,8,Albania,21111.01,"Meat of cattle with the bone, fresh or chilled"
...,...,...,...,...
38943,716,Zimbabwe,01510,Potatoes
38946,716,Zimbabwe,02211,Raw milk of cattle
38947,716,Zimbabwe,01802,Sugar cane
38950,716,Zimbabwe,01234,Tomatoes


Merge all data frames to 1.

In [28]:
def merging_price (df, df2010, df2011, df2012, df2013, df2014, df2015, df2016, df2017, df2018, df2019, df2020):
    z = pd.merge(df, df2010, how="left").merge(df2011, how="left").merge(df2012, how="left")
    z = pd.merge(z, df2013, how="left").merge(df2014, how="left").merge(df2015, how="left")
    z = pd.merge(z, df2016, how="left").merge(df2017, how="left").merge(df2018, how="left")        
    z = pd.merge(z, df2019, how="left").merge(df2020, how="left") 
    return z

In [29]:
df = merging_price (price, df2010, df2011, df2012, df2013, df2014, df2015, df2016, df2017, df2018, df2019, df2020)

In [30]:
df

,Area Code,Area,Item Code,Item,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,8,Albania,01341,Apples,356.0,386.5,369.7,492.1,474.8,389.0,381.3,461.6,435.2,346.7,510.3
1,8,Albania,0115,Barley,307.9,339.0,316.1,482.6,431.4,341.4,319.7,303.6,302.2,306.1,286.6
2,8,Albania,0231,"Hen eggs in shell, fresh",2405.3,2322.1,2165.6,1971.5,3143.7,3061.7,2946.5,3108.0,3358.5,3469.9,3511.9
3,8,Albania,0112,Maize (corn),288.6,386.5,360.5,416.4,353.9,293.7,290.2,278.7,300.3,277.0,283.2
4,8,Albania,21111.01,"Meat of cattle with the bone, fresh or chilled",6494.4,7572.2,6493.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2633,716,Zimbabwe,01510,Potatoes,NaN,NaN,NaN,NaN,NaN,620.0,635.0,625.0,NaN,NaN,NaN
2634,716,Zimbabwe,02211,Raw milk of cattle,NaN,NaN,NaN,NaN,450.0,NaN,NaN,NaN,NaN,NaN,NaN
2635,716,Zimbabwe,01802,Sugar cane,NaN,NaN,NaN,NaN,NaN,536.0,550.0,574.0,NaN,NaN,NaN
2636,716,Zimbabwe,01234,Tomatoes,NaN,NaN,NaN,NaN,NaN,504.0,567.0,786.0,NaN,NaN,NaN
